In [ ]:
import numpy as np

import qcodes as qc

## Multidimensional scanning module
from qcodes.dataset import (
    LinSweep,
    Measurement,
    dond,
    experiments,
    initialise_or_create_database_at,
    load_by_run_spec,
    load_or_create_experiment,
    plot_dataset,
)

## Using interactive widget
from qcodes.interactive_widget import experiments_widget

## Dummy instruments for generating synthetic data
from qcodes.tests.instrument_mocks import (
    DummyInstrument,
    DummyInstrumentWithMeasurement,
)
import qcodes

# A dummy signal generator with two parameters ch1 and ch2
dac = DummyInstrument('dac', gates=['ch'+str(f"{i}") for i in range(1,17)])

# A dummy digital multimeter that generates a synthetic data depending
# on the values set on the setter_instr, in this case the dummy dac
dmm = DummyInstrumentWithMeasurement('dmm', setter_instr=dac)

In [ ]:
from bootstrapper import Bootstrapper
import pandas as pd

Bootstrapper = Bootstrapper()
Bootstrapper.initialize_dut('DUT_nMOS.csv')
